<a href="https://colab.research.google.com/github/MariamMoawed/AI_Podcast_with_Agentic_RAG/blob/main/rag_document_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import faiss
import numpy as np
import requests
import os
from flask import Flask, request, jsonify
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Initialize Flask app
app = Flask(__name__)

# Load embedding model for text representation
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Example dialogues dataset
dialogues = [
    "Hello there! How are you today?",
    "I'm doing well, thank you! What about you?",
    "I'm great! Have you heard about the new AI model?",
    "Yes, it’s fascinating how AI is evolving!"
]

# Convert dialogues to embeddings
embeddings = np.array([embedding_model.encode(text) for text in dialogues])

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Function to retrieve relevant dialogues
def retrieve_relevant_dialogue(query, top_k=2):
    query_embedding = embedding_model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    return [dialogues[i] for i in indices[0]]

# Load Llama 3 (Placeholder: Use appropriate API or local model)
llama_pipeline = pipeline("text-generation", model="meta-llama/Llama-3")

def generate_dialogue(context):
    prompt = f"Character 1: {context[0]}\nCharacter 2: "
    response = llama_pipeline(prompt, max_length=50, num_return_sequences=1)
    return response[0]['generated_text']

# ElevenLabs API integration for voice synthesis
ELEVENLABS_API_KEY = "your_api_key_here"
ELEVENLABS_VOICE_ID = "your_voice_id_here"

def text_to_speech(text, filename="output.mp3"):
    url = "https://api.elevenlabs.io/v1/text-to-speech"
    headers = {"xi-api-key": ELEVENLABS_API_KEY, "Content-Type": "application/json"}
    payload = {"text": text, "voice_id": ELEVENLABS_VOICE_ID}
    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        with open(filename, "wb") as audio_file:
            audio_file.write(response.content)
        print(f"Audio saved as {filename}")
    else:
        print("Error in voice synthesis", response.text)

# API Endpoint to process user query
@app.route("/generate", methods=["POST"])
def generate():
    data = request.json
    query = data.get("query", "")

    retrieved_context = retrieve_relevant_dialogue(query)
    generated_response = generate_dialogue(retrieved_context)
    text_to_speech(generated_response, "output.mp3")

    return jsonify({
        "retrieved_context": retrieved_context,
        "generated_response": generated_response,
        "audio_file": "output.mp3"
    })

# Run the application
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)
